In [2]:
from openai import OpenAI
import os
from dotenv import load_dotenv, find_dotenv
import utils_zh

_ = load_dotenv(find_dotenv())

client = OpenAI(
    # This is the default and can be omitted
    api_key=os.environ.get("OPENAI_API_KEY"),
    )

# langchain是一种快速开发应用程序框架，组件可以链式组合

In [3]:
from chat_tools import get_completion

In [4]:
get_completion("1+1是什么？")

'1+1等于2。'

#### 转换邮件语气

In [5]:
# 非正式用语
customer_email = """   
阿，我很生气，\
因为我的搅拌机盖掉了，\
把奶昔溅到了厨房的墙上！\
更糟糕的是，保修不包括打扫厨房的费用。\
我现在需要你的帮助，伙计！
"""

In [6]:
# 普通话 + 平静、尊敬的语调
customer_style = """正式普通话 \
用一个平静、尊敬的语调
"""

# 要求模型根据给出的语调进行转化
prompt = f"""把由三个反引号分隔的文本\
翻译成一种{customer_style}风格。
文本: ```{customer_email}```
"""
response = get_completion(prompt)

In [7]:
response

'尊敬的先生/女士，我感到非常不安，因为我的搅拌机盖子不慎掉落，导致奶昔溅到了厨房的墙壁上！更令人困扰的是，保修并不包括清洁厨房的费用。我现在真的需要您的帮助，朋友！感谢您的理解和支持。'

#### 通过langchain使用OpenAI

In [8]:
from langchain_community.chat_models.openai import ChatOpenAI

chat = ChatOpenAI(temperature = 0)

In [9]:
# 要求模型根据给出的语调进行转化

template_string = """Translate the text \
that is delimited by triple backticks \
into a style that is {style}. \
text: ```{text}```
"""

from langchain.prompts.chat import ChatPromptTemplate
#构造prompt模板
prompt_template = ChatPromptTemplate.from_template(template_string)

print(prompt_template.messages[0].prompt)

input_variables=['style', 'text'] template='Translate the text that is delimited by triple backticks into a style that is {style}. text: ```{text}```\n'


In [20]:
print(prompt_template.messages[0].prompt.input_variables)

['style', 'text']


In [23]:
customer_messages = prompt_template.format_messages(
                    style=customer_style,
                    text=customer_email)

print(type(customer_messages))
print(type(customer_messages[0]))
print(customer_messages[0])


<class 'list'>
<class 'langchain_core.messages.human.HumanMessage'>
content='Translate the text that is delimited by triple backticks into a style that is 正式普通话 用一个平静、尊敬的语调\n. text: ```   \n阿，我很生气，因为我的搅拌机盖掉了，把奶昔溅到了厨房的墙上！更糟糕的是，保修不包括打扫厨房的费用。我现在需要你的帮助，伙计！\n```\n'


In [24]:
# 中文提示
from langchain.prompts import ChatPromptTemplate

template_string = """把由三个反引号分隔的文本\
翻译成一种{style}风格。\
文本: ```{text}```
"""
prompt_template = ChatPromptTemplate.from_template(template_string)

print(prompt_template.messages[0].prompt.input_variables)


customer_style = """正式普通话 \
用一个平静、尊敬的语气
"""

customer_email = """
阿，我很生气，\
因为我的搅拌机盖掉了，\
把奶昔溅到了厨房的墙上！\
更糟糕的是，保修不包括打扫厨房的费用。\
我现在需要你的帮助，伙计！
"""

customer_messages = prompt_template.format_messages(
                    style=customer_style,
                    text=customer_email)


print(customer_messages[0])

['style', 'text']
content='把由三个反引号分隔的文本翻译成一种正式普通话 用一个平静、尊敬的语气\n风格。文本: ```\n阿，我很生气，因为我的搅拌机盖掉了，把奶昔溅到了厨房的墙上！更糟糕的是，保修不包括打扫厨房的费用。我现在需要你的帮助，伙计！\n```\n'


In [25]:
customer_response = chat.invoke(customer_messages, temperature=0)
print(customer_response.content)

非常抱歉听到您的困扰。我理解您的不快，因为搅拌机盖掉了，导致奶昔溅到了厨房的墙上。更令人不安的是，保修不包括打扫厨房的费用。在这种情况下，我愿意提供帮助，尽我所能解决问题。请告诉我您需要我做什么，我会尽力协助您解决这个困难。感谢您的信任和合作。


In [28]:
service_reply = """嘿，顾客， \
保修不包括厨房的清洁费用， \
因为您在启动搅拌机之前 \
忘记盖上盖子而误用搅拌机, \
这是您的错。 \
倒霉！ 再见！
"""
service_style_pirate = """\
一个有礼貌的语气 \
使用正式的普通话 \
"""

service_messages = prompt_template.format_messages(
    style=service_style_pirate,
    text=service_reply)

print(service_messages[0].content)

service_response = chat.invoke(service_messages, temperature=0)
print(service_response.content)

把由三个反引号分隔的文本翻译成一种一个有礼貌的语气 使用正式的普通话 风格。文本: ```嘿，顾客， 保修不包括厨房的清洁费用， 因为您在启动搅拌机之前 忘记盖上盖子而误用搅拌机, 这是您的错。 倒霉！ 再见！
```

尊敬的客户，根据保修条款，很遗憾地通知您，厨房清洁费用不在保修范围内。这是因为在使用搅拌机之前，您忘记盖上盖子，导致搅拌机误用。希望您能理解这一点。祝您好运！再见！感谢您的合作。


In [29]:
# 中文版
prompt = """ 你的任务是判断学生的解决方案是正确的还是不正确的

要解决该问题，请执行以下操作：
 - 首先，制定自己的问题解决方案
 - 然后将您的解决方案与学生的解决方案进行比较
 并评估学生的解决方案是否正确。
...
使用下面的格式:

问题:
```
问题文本
```
学生的解决方案:
```
学生的解决方案文本
```
实际解决方案:
```
...
制定解决方案的步骤以及您的解决方案请参见此处
```
学生的解决方案和实际解决方案是否相同 \
只计算：
```
是或者不是
```
学生的成绩
```
正确或者不正确
```

问题:
```
{question}
```
学生的解决方案:
```
{student's solution}
```
实际解决方案:

"""

In [30]:
from langchain.prompts import ChatPromptTemplate

customer_review = """
这款吹叶机非常神奇。 它有四个设置：\
吹蜡烛、微风、风城、龙卷风。 \
两天后就到了，正好赶上我妻子的\
周年纪念礼物。 \
我想我的妻子会喜欢它到说不出话来。 \
到目前为止，我是唯一一个使用它的人，而且我一直\
每隔一天早上用它来清理草坪上的叶子。 \
它比其他吹叶机稍微贵一点，\
但我认为它的额外功能是值得的。
"""

review_template = """\
对于以下文本，请从中提取以下信息：

礼物：该商品是作为礼物送给别人的吗？ \
如果是，则回答 是的；如果否或未知，则回答 不是。

交货天数：产品需要多少天\
到达？ 如果没有找到该信息，则输出-1。

价钱：提取有关价值或价格的任何句子，\
并将它们输出为逗号分隔的 Python 列表。

使用以下键将输出格式化为 JSON：
礼物
交货天数
价钱

文本: {text}
"""
prompt_template = ChatPromptTemplate.from_template(review_template)
print(prompt_template)

input_variables=['text'] messages=[HumanMessagePromptTemplate(prompt=PromptTemplate(input_variables=['text'], template='对于以下文本，请从中提取以下信息：\n\n礼物：该商品是作为礼物送给别人的吗？ 如果是，则回答 是的；如果否或未知，则回答 不是。\n\n交货天数：产品需要多少天到达？ 如果没有找到该信息，则输出-1。\n\n价钱：提取有关价值或价格的任何句子，并将它们输出为逗号分隔的 Python 列表。\n\n使用以下键将输出格式化为 JSON：\n礼物\n交货天数\n价钱\n\n文本: {text}\n'))]


In [31]:
prompt_template.input_variables

['text']

In [32]:
messages = prompt_template.format_messages(text=customer_review)

chat = ChatOpenAI(temperature=0.0)
response = chat.invoke(messages)
print(response.content)

{
    "礼物": "是的",
    "交货天数": 2,
    "价钱": ["它比其他吹叶机稍微贵一点"]
}


In [33]:
review_template_2 = """\
对于以下文本，请从中提取以下信息：：

礼物：该商品是作为礼物送给别人的吗？
如果是，则回答 是的；如果否或未知，则回答 不是。

交货天数：产品到达需要多少天？ 如果没有找到该信息，则输出-1。

价钱：提取有关价值或价格的任何句子，并将它们输出为逗号分隔的 Python 列表。

文本: {text}

{format_instructions}
"""

In [34]:
from langchain.output_parsers import ResponseSchema
from langchain.output_parsers import StructuredOutputParser

gift_schema = ResponseSchema(name="礼物",
                             description="这件物品是作为礼物送给别人的吗？\
                            如果是，则回答 是的，\
                            如果否或未知，则回答 不是。")

delivery_days_schema = ResponseSchema(name="交货天数",
                                      description="产品需要多少天才能到达？\
                                      如果没有找到该信息，则输出-1。")

price_value_schema = ResponseSchema(name="价钱",
                                    description="提取有关价值或价格的任何句子，\
                                    并将它们输出为逗号分隔的 Python 列表")

response_schemas = [gift_schema,
                    delivery_days_schema,
                    price_value_schema]
output_parser = StructuredOutputParser.from_response_schemas(response_schemas)
format_instructions = output_parser.get_format_instructions()
print(format_instructions)

The output should be a markdown code snippet formatted in the following schema, including the leading and trailing "```json" and "```":

```json
{
	"礼物": string  // 这件物品是作为礼物送给别人的吗？                            如果是，则回答 是的，                            如果否或未知，则回答 不是。
	"交货天数": string  // 产品需要多少天才能到达？                                      如果没有找到该信息，则输出-1。
	"价钱": string  // 提取有关价值或价格的任何句子，                                    并将它们输出为逗号分隔的 Python 列表
}
```


In [36]:
prompt_template = ChatPromptTemplate.from_template(review_template_2)
messages = prompt_template.format_messages(text=customer_review, format_instructions=format_instructions)
print(messages[0].content)

对于以下文本，请从中提取以下信息：：

礼物：该商品是作为礼物送给别人的吗？
如果是，则回答 是的；如果否或未知，则回答 不是。

交货天数：产品到达需要多少天？ 如果没有找到该信息，则输出-1。

价钱：提取有关价值或价格的任何句子，并将它们输出为逗号分隔的 Python 列表。

文本: 
这款吹叶机非常神奇。 它有四个设置：吹蜡烛、微风、风城、龙卷风。 两天后就到了，正好赶上我妻子的周年纪念礼物。 我想我的妻子会喜欢它到说不出话来。 到目前为止，我是唯一一个使用它的人，而且我一直每隔一天早上用它来清理草坪上的叶子。 它比其他吹叶机稍微贵一点，但我认为它的额外功能是值得的。


The output should be a markdown code snippet formatted in the following schema, including the leading and trailing "```json" and "```":

```json
{
	"礼物": string  // 这件物品是作为礼物送给别人的吗？                            如果是，则回答 是的，                            如果否或未知，则回答 不是。
	"交货天数": string  // 产品需要多少天才能到达？                                      如果没有找到该信息，则输出-1。
	"价钱": string  // 提取有关价值或价格的任何句子，                                    并将它们输出为逗号分隔的 Python 列表
}
```



In [37]:
response = chat.invoke(messages)
print(response.content)

```json
{
	"礼物": "是的",
	"交货天数": "两天后",
	"价钱": "它比其他吹叶机稍微贵一点"
}
```


In [38]:
output_dict = output_parser.parse(response.content)
output_dict

{'礼物': '是的', '交货天数': '两天后', '价钱': '它比其他吹叶机稍微贵一点'}

#### 链式思考推理（agent使用）

In [41]:
# !pip install wikipedia

In [6]:
from langchain.docstore.wikipedia import Wikipedia
from langchain_community.chat_models import ChatOpenAI
from langchain.agents import initialize_agent, Tool, AgentExecutor
from langchain.agents.react.base import DocstoreExplorer

docstore=DocstoreExplorer(Wikipedia())
tools = [
  Tool(
    name="Search",
    func=docstore.search,
    description="Search for a term in the docstore.",
  ),
  Tool(
    name="Lookup",
    func=docstore.lookup,
    description="Lookup a term in the docstore.",
  )
]

# 使用大语言模型
llm = ChatOpenAI(
  model_name="gpt-3.5-turbo",
  temperature=0,
)

# 初始化ReAct代理
react = initialize_agent(tools, llm, agent="react-docstore", verbose=True)
agent_executor = AgentExecutor.from_agent_and_tools(
  agent=react.agent,
  tools=tools,
  verbose=True,
  #
  handle_parsing_errors=True
)


question = "Author David Chanoff has collaborated with a U.S. Navy admiral who served as the ambassador to the United Kingdom under which President?"
agent_executor.run(question)



> Entering new AgentExecutor chain...
Thought: I need to search David Chanoff, find the U.S. Navy admiral he collaborated with, and then find out which President the admiral served as the ambassador to the United Kingdom under.
Action: Search[David Chanoff]
Observation: David Chanoff (born November 15, 1943, in Philadelphia) is an American author of non-fiction work. His work has typically involved collaborations with the principal protagonist of the work concerned. His collaborators have included Augustus A. White, Joycelyn Elders, Đoàn Văn Toại, William J. Crowe, Ariel Sharon, Kenneth Good and Felix Zandman, among others. He has also written about a wide range of subjects including literary history, education and foreign for The Washington Post, The New Republic and The New York Times Magazine.
Chanoff founded Sudbury Valley School in Framingham, Massachusetts in 1968. In the late 1970s and early 1980s, he taught English at Tufts University and Harvard University. 
In 2015, Chanoff

'George H. W. Bush'